In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [4]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

## train-test-split manually.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

## Scaling

In [6]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
X_train

array([[-0.42084864, -1.30090917, -0.43998714, ..., -0.79353449,
        -0.83540656, -0.82067768],
       [-0.96169845,  0.49991949, -1.00025835, ..., -1.77287597,
        -0.6873757 , -1.00538948],
       [-1.72964703,  0.31749789, -1.6437047 , ..., -0.55638038,
         0.74968794,  3.62248077],
       ...,
       [-0.80176975, -1.99317577, -0.79633145, ..., -0.38978579,
         0.2407279 ,  0.07545443],
       [-0.10680681, -0.83082272, -0.07477644, ...,  0.3624374 ,
        -0.46948758,  1.0700995 ],
       [-1.0460245 ,  0.20523844, -0.93101609, ...,  0.48904311,
        -0.45618143,  2.02164515]])

## Label Encoding

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

## Numpy arrays to PyTorch tensors

In [9]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [10]:
X_train_tensor.shape

torch.Size([455, 30])

In [11]:
y_train_tensor.shape

torch.Size([455])

# Define Model manually

In [12]:
class MySimpleNN():

  def __init__(self, X):

    # This is constructor
    # torch.rand(X.shape[1] means columns or features of X dataset i.e if dataset has 30 features then weights dimension will have (30, 1) dimension. 1 is coming from (X.shape[1], 1).
    # You can print out weights to check dimension in Training Pipeline function.
    self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
    # print(weights)

    # torch.zeros(1, means bias will have scalar / single value i.e (0). You can check bias to check dimension in Training Pipeline function.
    self.bias = torch.zeros(1, dtype=torch.float64, requires_grad=True)
    # print(bias)


  def forward(self, X):
    z = torch.matmul(X, self.weights) + self.bias
    y_pred = torch.sigmoid(z)
    return y_pred

  def loss_function(self, y_pred, y):

    #(Optional): Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

    # Calculate loss
    loss = -(y_train_tensor * torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
    return loss



## Important parameters

In [13]:
learning_rate = 0.1
epochs = 25

# Training Pipeline

In [15]:
# create model
model = MySimpleNN(X_train_tensor)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculate
  loss = model.loss_function(y_pred, y_train_tensor)

  # backward pass ( BackPropagation )
  loss.backward()

  # parameters update
  # torch.no_grad means, we are not tracking weights and bias values. Just update the weights and bias values.
  with torch.no_grad():
    model.weights -= learning_rate * model.weights.grad
    model.bias -= learning_rate * model.bias.grad

  # zero gradients
  # we are doing this, to clear grad history to get accurate values.
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.681739959160112
Epoch: 2, Loss: 3.5542999199975296
Epoch: 3, Loss: 3.4200819742838666
Epoch: 4, Loss: 3.280529204746359
Epoch: 5, Loss: 3.135417629184805
Epoch: 6, Loss: 2.982730554304207
Epoch: 7, Loss: 2.827770092701493
Epoch: 8, Loss: 2.6666160588715955
Epoch: 9, Loss: 2.502278651616507
Epoch: 10, Loss: 2.3392955703580594
Epoch: 11, Loss: 2.174485365111346
Epoch: 12, Loss: 2.01448706453363
Epoch: 13, Loss: 1.8596261709873854
Epoch: 14, Loss: 1.709809939303379
Epoch: 15, Loss: 1.569338372846991
Epoch: 16, Loss: 1.4381113128043164
Epoch: 17, Loss: 1.3210458355191272
Epoch: 18, Loss: 1.2195304222186085
Epoch: 19, Loss: 1.134311001569275
Epoch: 20, Loss: 1.0650377848216706
Epoch: 21, Loss: 1.0102149230485118
Epoch: 22, Loss: 0.9675502649816129
Epoch: 23, Loss: 0.9344067553975728
Epoch: 24, Loss: 0.9082551987616807
Epoch: 25, Loss: 0.8870239213232697


In [16]:
model.weights

tensor([[ 0.2149],
        [ 0.4702],
        [-0.1725],
        [ 0.4882],
        [ 0.3178],
        [ 0.0372],
        [-0.1864],
        [ 0.0009],
        [ 0.0449],
        [-0.1239],
        [-0.3201],
        [ 0.5192],
        [ 0.3244],
        [-0.3203],
        [ 0.3563],
        [-0.0798],
        [-0.4023],
        [ 0.4128],
        [ 0.0668],
        [ 0.4193],
        [-0.3870],
        [ 0.3583],
        [ 0.1300],
        [ 0.1326],
        [ 0.1904],
        [-0.0550],
        [-0.0594],
        [-0.3358],
        [ 0.1662],
        [-0.1622]], dtype=torch.float64, requires_grad=True)

In [17]:
model.bias

tensor([-0.1752], dtype=torch.float64, requires_grad=True)

## Evaluation

In [18]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.9).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')


Accuracy: 0.6154201030731201
